In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
# from tqdm import tqdm
from tqdm.notebook import tqdm
import warnings
import re
import csv
import os
from datetime import datetime
import glob
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from xml.etree.ElementTree import fromstring 
import shutil
import warnings
import random

In [3]:
# Remove old data and database

if os.path.exists("data"):
    shutil.rmtree("data")
if os.path.exists("ufc_database.db"):
    os.remove("ufc_database.db")
os.makedirs("data")

# # Create required directories
# directories = ['./data/fighters/', './data/git/']
# for directory in directories:
#     os.makedirs(directory, exist_ok=True)

# # Define CSV files and their columns
# csv_files = {
#     './data/event_urls_sherdog.csv': ['Event_URL'],
#     './data/fighter_id_sherdog.csv': ['Fighter', 'Fighter_ID'],
#     './data/event_data_sherdog.csv': ['Event Name', 'Event Location', 'Event Date', 
#                                       'Fighter 1', 'Fighter 2', 'Weight Class', 
#                                       'Winning Fighter', 'Winning Method', 
#                                       'Winning Round', 'Winning Time', 'Referee'],
#     './data/fighter_info.csv': ['Fighter', 'Nickname', 'Birth Date', 'Nationality', 
#                                 'Hometown', 'Association', 'Weight Class', 'Height', 
#                                 'REACH', 'STANCE', 'Wins', 'Losses', 'Win_Decision', 
#                                 'Win_KO', 'Win_Sub', 'Loss_Decision', 'Loss_KO', 
#                                 'Loss_Sub', 'Sherdog URL', 'BFO URL']
# }

# # Ensure CSV files exist and have headers
# for file_path, columns in csv_files.items():
#     if not os.path.exists(file_path):
#         pd.DataFrame(columns=columns).to_csv(file_path, index=False)

# # Function to download GitHub data
# def download_github_files():
#     files = ["ufc_event_details.csv", "ufc_fight_results.csv", 
#              "ufc_fight_stats.csv", "ufc_fighter_tott.csv"]
#     base_url = "https://raw.githubusercontent.com/Greco1899/scrape_ufc_stats/main/"
#     save_path = "./data/git/"

#     for file in files:
#         response = requests.get(base_url + file)
#         if response.status_code == 200:
#             with open(os.path.join(save_path, file), 'wb') as f:
#                 f.write(response.content)
#         else:
#             print(f"Failed to download {file}")

# # Execute file download
# download_github_files()


In [4]:
# Sherdog: Event URLs
# event_urls_sherdog.csv

urls = [
    'https://www.sherdog.com/organizations/Ultimate-Fighting-Championship-UFC-2/recent-events/1',
    'https://www.sherdog.com/organizations/Ultimate-Fighting-Championship-UFC-2/recent-events/2',
    'https://www.sherdog.com/organizations/Ultimate-Fighting-Championship-UFC-2/recent-events/3',
    'https://www.sherdog.com/organizations/Ultimate-Fighting-Championship-UFC-2/recent-events/4',
    'https://www.sherdog.com/organizations/Ultimate-Fighting-Championship-UFC-2/recent-events/5',
    'https://www.sherdog.com/organizations/Ultimate-Fighting-Championship-UFC-2/recent-events/6',
    'https://www.sherdog.com/organizations/Ultimate-Fighting-Championship-UFC-2/recent-events/7'
]
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/121.0"
]
headers = {"User-Agent": random.choice(user_agents)}
df = pd.read_csv('./data/event_urls_sherdog.csv') if os.path.isfile('./data/event_urls_sherdog.csv') else pd.DataFrame(columns=['Event_URL'])
existing_urls = set(df['Event_URL'])

print(f"Starting to scrape {len(urls)} URLs...")
for i, url in enumerate(urls, 1):
    print(f"Processing URL {i}/{len(urls)}")
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        specific_div = soup.find('div', {'class': 'single_tab', 'id': 'recent_tab'})
        if specific_div:
            new_urls = [a.get('href') for a in specific_div.find_all('a', itemprop='url') if a.get('href') and a.get('href') not in existing_urls]
            df = pd.concat([df, pd.DataFrame(new_urls, columns=['Event_URL'])], ignore_index=True)
    except requests.RequestException:
        print(f"Failed to process URL {i}")
        pass

df.to_csv('./data/event_urls_sherdog.csv', index=False)
print("Updated event URLs saved.")

# Remove Broken URL's 
urls_to_delete = {
    "/events/UFC-233-Ultimate-Fighting-Championship-233-72021",
    "/events/UFC-Fight-Night-97-Lamas-vs-Penn-90890",
    "/events/UFC-176-Aldo-vs-Mendes-2-37609",
    "/events/UFC-151-Jones-vs-Henderson-25809"
}
df = pd.read_csv("./data/event_urls_sherdog.csv")
df = df[~df["Event_URL"].isin(urls_to_delete)]
df.to_csv("./data/event_urls_sherdog.csv", index=False)
print("Broken URLs removed.")

# Event URLs 
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
]
df = pd.read_csv('./data/event_urls_sherdog.csv')
df['Event_URL'] = "https://sherdog.com" + df['Event_URL'].astype(str)
event_dates = []
with requests.Session() as session:
    session.headers.update({"User-Agent": random.choice(user_agents)})
    with ThreadPoolExecutor(max_workers=40) as executor:
        # futures = {executor.submit(session.get, url, {"User-Agent": random.choice(user_agents)}, 30): url for url in df['Event_URL']}
        futures = {executor.submit(lambda url: session.get(url, headers={"User-Agent": random.choice(user_agents)}, timeout=30), url): url for url in df['Event_URL']}
        for future in tqdm(as_completed(futures), total=len(df['Event_URL']), desc="Scraping Events", unit="Event"):
            url = futures[future]
            try:
                response = future.result()
                if response.status_code == 200:
                    soup = BeautifulSoup(response.content, 'html.parser')
                    event_date_meta = soup.find('meta', itemprop='startDate')
                    event_dates.append(event_date_meta['content'].strip() if event_date_meta else None)
                else:
                    event_dates.append("Error")
            except requests.exceptions.RequestException:
                event_dates.append("Error")
df['Event Date'] = event_dates
df.to_csv('./data/event_urls_sherdog.csv', index=False)
print("Event dates appended successfully.")
print(f"Total number of rows including the header in event_urls_sherdog.csv: {len(df)}")
print(f"Column names: {list(df.columns)}")


Starting to scrape 7 URLs...
Processing URL 1/7
Processing URL 2/7
Processing URL 3/7
Processing URL 4/7
Processing URL 5/7
Processing URL 6/7
Processing URL 7/7
Updated event URLs saved.
Broken URLs removed.


Scraping Events:   0%|          | 0/696 [00:00<?, ?Event/s]

Event dates appended successfully.
Total number of rows including the header in event_urls_sherdog.csv: 696
Column names: ['Event_URL', 'Event Date']


In [5]:
# Sherdog: Event Data
# event_data_sherdog.csv

warnings.filterwarnings("ignore", category=FutureWarning)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}
urls_df = pd.read_csv('data/event_urls_sherdog.csv')
all_data = []
def fetch_event_data(url, session):
    full_url = f'https://sherdog.com{url}' if not url.startswith('http') else url
    event_data = []
    try:
        with session.get(full_url, headers=headers) as response:
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                event_name = soup.find('span', itemprop='name').text.strip()
                event_location = soup.find('span', itemprop='location').text.strip()
                event_date = soup.find('meta', itemprop='startDate')['content'].strip()
                main_event_fighters = soup.find_all('div', class_='fighter')
                if main_event_fighters:
                    fighter1 = main_event_fighters[0].find('span', itemprop='name').text.strip()
                    fighter2 = main_event_fighters[1].find('span', itemprop='name').text.strip()
                    fighter1_id = main_event_fighters[0].find('a', itemprop='url')['href'].split('-')[-1]
                    fighter2_id = main_event_fighters[1].find('a', itemprop='url')['href'].split('-')[-1]
                    weight_class = soup.find('span', class_='weight_class').text.strip()
                    winning_fighter = fighter1  
                    winning_method_em = soup.find('em', string='Method').parent
                    winning_method = winning_method_em.contents[2].strip()
                    winning_round_em = soup.find('em', string='Round').parent
                    winning_round = winning_round_em.contents[2].strip()
                    winning_time_em = soup.find('em', string='Time').parent
                    winning_time = winning_time_em.contents[2].strip()
                    referee_em = soup.find('em', string='Referee').parent
                    referee = referee_em.find('a').text.strip()
                    event_data.append({
                        'Event Name': event_name,
                        'Event Location': event_location,
                        'Event Date': event_date,
                        'Fighter 1': fighter1,
                        'Fighter 2': fighter2,
                        'Fighter 1 ID': fighter1_id,
                        'Fighter 2 ID': fighter2_id,
                        'Weight Class': weight_class,
                        'Winning Fighter': winning_fighter,
                        'Winning Method': winning_method,
                        'Winning Round': winning_round,
                        'Winning Time': winning_time,
                        'Referee': referee,
                        'Fight Type': 'Main Event'
                    })
                other_bouts = soup.find_all('tr', itemprop='subEvent')
                for bout in other_bouts:
                    fighters = bout.find_all('div', class_='fighter_list')
                    if len(fighters) >= 2:
                        fighter1 = fighters[0].find('img')['title']
                        fighter2 = fighters[1].find('img')['title']
                        fighter1_url = fighters[0].find('a', itemprop='url')['href']
                        fighter2_url = fighters[1].find('a', itemprop='url')['href']
                        fighter1_id = fighter1_url.split('-')[-1]
                        fighter2_id = fighter2_url.split('-')[-1]
                        weight_class = bout.find('span', class_='weight_class')
                        weight_class = weight_class.text.strip() if weight_class else "Unknown"
                        winning_method = bout.find('td', class_='winby').find('b').get_text(strip=True)
                        winning_round = bout.find_all('td')[-2].get_text(strip=True)
                        winning_time = bout.find_all('td')[-1].get_text(strip=True)
                        referee = bout.find('td', class_='winby').find('a').get_text(strip=True)
                        event_data.append({
                            'Event Name': event_name,
                            'Event Location': event_location,
                            'Event Date': event_date,
                            'Fighter 1': fighter1,
                            'Fighter 2': fighter2,
                            'Fighter 1 ID': fighter1_id,
                            'Fighter 2 ID': fighter2_id,
                            'Weight Class': weight_class,
                            'Winning Fighter': fighter1,  
                            'Winning Method': winning_method,
                            'Winning Round': winning_round,
                            'Winning Time': winning_time,
                            'Referee': referee,
                            'Fight Type': 'Undercard'
                        })
        return event_data
    except Exception as e:
        print(f"Request failed for {full_url}: {e}")
        return None
session = requests.Session()
total_urls = len(urls_df['Event_URL'])
completed_requests = 0
with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
    futures = [executor.submit(fetch_event_data, url, session) for url in urls_df['Event_URL']]
    for future in concurrent.futures.as_completed(futures):
        data = future.result()
        completed_requests += 1
        progress_percentage = (completed_requests / total_urls) * 100
        print(f"Completed {completed_requests}/{total_urls} requests ({progress_percentage:.2f}%)")
        if data:
            all_data.extend(data)
df = pd.DataFrame(all_data)
file_path = './data/event_data_sherdog.csv'
write_mode = 'a' if os.path.isfile(file_path) else 'w'
df.to_csv(file_path, mode=write_mode, header=not os.path.isfile(file_path), index=False)
print("Data successfully written to data/event_data_sherdog.csv")
print(f"Total number of rows: {len(df)}")
print(f"Column names: {list(df.columns)}")
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df.to_csv(file_path, index=False)

Completed 1/696 requests (0.14%)
Completed 2/696 requests (0.29%)
Completed 3/696 requests (0.43%)
Completed 4/696 requests (0.57%)
Completed 5/696 requests (0.72%)
Completed 6/696 requests (0.86%)
Completed 7/696 requests (1.01%)
Completed 8/696 requests (1.15%)
Completed 9/696 requests (1.29%)
Completed 10/696 requests (1.44%)
Completed 11/696 requests (1.58%)
Completed 12/696 requests (1.72%)
Completed 13/696 requests (1.87%)
Completed 14/696 requests (2.01%)
Completed 15/696 requests (2.16%)
Completed 16/696 requests (2.30%)
Completed 17/696 requests (2.44%)
Completed 18/696 requests (2.59%)
Completed 19/696 requests (2.73%)
Completed 20/696 requests (2.87%)
Completed 21/696 requests (3.02%)
Completed 22/696 requests (3.16%)
Completed 23/696 requests (3.30%)
Completed 24/696 requests (3.45%)
Completed 25/696 requests (3.59%)
Completed 26/696 requests (3.74%)
Completed 27/696 requests (3.88%)
Completed 28/696 requests (4.02%)
Completed 29/696 requests (4.17%)
Completed 30/696 reques

In [15]:
# Sherdog: Fighter IDs
# fighter_id_sherdog.csv

df = pd.read_csv('./data/event_data_sherdog.csv')
df2 = pd.DataFrame(columns=['Fighter', 'Fighter_ID'])
df2.to_csv('./data/fighter_id_sherdog.csv', index=False)
for index, row in df.iterrows():
    fighter1 = row['Fighter 1']
    fighter2 = row['Fighter 2']
    fighter1_id = row['Fighter 1 ID']
    fighter2_id = row['Fighter 2 ID']
    for fighter, fighter_id in zip([fighter1, fighter2], [fighter1_id, fighter2_id]):
        if fighter not in df2['Fighter'].values and fighter_id not in df2['Fighter_ID'].values:
            df2 = pd.concat([df2, pd.DataFrame([{'Fighter': fighter, 'Fighter_ID': fighter_id}])])  # adjusted line
df2.to_csv('./data/fighter_id_sherdog.csv', index=False)
print("Data successfully written to data/fighter_id_sherdog.csv")
print(f"Total number of rows: {len(df2)}")
print(f"Column names: {list(df2.columns)}")

# Remove nicknames
# df = pd.read_csv('./data/fighter_id_sherdog.csv')
# df['Fighter'] = df['Fighter'].str.replace(r" '.+?'", "", regex=True)
# df.to_csv('./data/fighter_id_sherdog.csv', index=False)
# df = pd.read_csv('./data/event_data_sherdog.csv')
# for col in ['Fighter 1', 'Fighter 2', 'Winning Fighter']:
#     df[col] = df[col].str.replace(r" '.+?'", "", regex=True)
# df.to_csv('./data/event_data_sherdog.csv', index=False)

# Add 'UFC' indicator to current fighters in fighter_id_sherdog.csv.csv
df = pd.read_csv('./data/fighter_id_sherdog.csv')
df['UFC'] = 'y'
df.to_csv('./data/fighter_id_sherdog.csv', index=False)
print("New column 'UFC' added to the CSV file.")


Data successfully written to data/fighter_id_sherdog.csv
Total number of rows: 2419
Column names: ['Fighter', 'Fighter_ID']
New column 'UFC' added to the CSV file.


In [ ]:



# df = pd.read_csv('./data/event_data_sherdog.csv')
# df2 = pd.read_csv('./data/fighter_id_sherdog.csv')

# # Iterate over each row in df
# for index, row in df.iterrows():
#     fighter1 = row['Fighter 1']
#     fighter2 = row['Fighter 2']
#     fighter1_id = row['Fighter 1 ID']
#     fighter2_id = row['Fighter 2 ID']
#     for fighter, fighter_id in zip([fighter1, fighter2], [fighter1_id, fighter2_id]):
#         if fighter not in df2['Fighter'].values and fighter_id not in df2['Fighter_ID'].values:
#             df2 = pd.concat([df2, pd.DataFrame([{'Fighter': fighter, 'Fighter_ID': fighter_id}])])  # adjusted line

# # Save the DataFrame
# df2.to_csv('./data/fighter_id_sherdog.csv', index=False)




In [ ]:
import requests
import pandas as pd
import os
from tqdm import tqdm
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed

warnings.filterwarnings("ignore", category=FutureWarning)
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

df = pd.read_csv('data/event_urls_sherdog.csv')
df['Event_URL'] = "https://sherdog.com" + df['Event_URL'].astype(str)
all_data, session = [], requests.Session()

with ThreadPoolExecutor(max_workers=30) as executor:
    futures = {executor.submit(session.get, url, headers=headers): url for url in df['Event_URL']}
    
    for future in tqdm(as_completed(futures), total=len(df['Event_URL']), desc="Scraping Event Data", unit="Event"):
        url = futures[future]
        try:
            response = future.result()
            if response.status_code != 200:
                continue
            
            soup = BeautifulSoup(response.content, 'html.parser')
            event_name = soup.find('span', itemprop='name').text.strip()
            event_location = soup.find('span', itemprop='location').text.strip()
            event_date = soup.find('meta', itemprop='startDate')['content'].strip()
            
            for bout in soup.find_all(['tr', 'div'], itemprop=['subEvent', 'fighter']):
                fighters = bout.find_all('div', class_='fighter_list') or bout.find_all('div', class_='fighter')
                if len(fighters) < 2:
                    continue
                
                fighter1 = fighters[0].find('img')['title'] if bout.name == 'tr' else fighters[0].find('span', itemprop='name').text.strip()
                fighter2 = fighters[1].find('img')['title'] if bout.name == 'tr' else fighters[1].find('span', itemprop='name').text.strip()
                fighter1_id = fighters[0].find('a', itemprop='url')['href'].split('-')[-1]
                fighter2_id = fighters[1].find('a', itemprop='url')['href'].split('-')[-1]
                weight_class = (bout.find('span', class_='weight_class') or {}).get_text(strip=True) or "Unknown"
                
                winning_method = (bout.find('td', class_='winby') or {}).find('b')
                winning_round = (bout.find_all('td')[-2] if bout.name == 'tr' else soup.find('em', string='Round').parent.contents[2])
                winning_time = (bout.find_all('td')[-1] if bout.name == 'tr' else soup.find('em', string='Time').parent.contents[2])
                referee = (bout.find('td', class_='winby') or {}).find('a')
                
                all_data.append({
                    'Event Name': event_name,
                    'Event Location': event_location,
                    'Event Date': event_date,
                    'Fighter 1': fighter1,
                    'Fighter 2': fighter2,
                    'Fighter 1 ID': fighter1_id,
                    'Fighter 2 ID': fighter2_id,
                    'Weight Class': weight_class,
                    'Winning Fighter': fighter1,
                    'Winning Method': winning_method.get_text(strip=True) if winning_method else "Unknown",
                    'Winning Round': winning_round.get_text(strip=True) if hasattr(winning_round, 'get_text') else winning_round.strip(),
                    'Winning Time': winning_time.get_text(strip=True) if hasattr(winning_time, 'get_text') else winning_time.strip(),
                    'Referee': referee.get_text(strip=True) if referee else "Unknown",
                    'Fight Type': 'Main Event' if bout.name == 'div' else 'Undercard'
                })
        except Exception as e:
            print(f"Request failed for {url}: {e}")

if all_data:
    pd.DataFrame(all_data).to_csv('./data/event_data_sherdog.csv', mode='a' if os.path.isfile('./data/event_data_sherdog.csv') else 'w', header=not os.path.isfile('./data/event_data_sherdog.csv'), index=False)
    print("Event data saved successfully.")

print(f"Total number of rows: {len(df)}")
print(f"Column names: {list(df.columns)}")


Scraping Event Data:   0%|          | 0/696 [00:13<?, ?Event/s]


In [7]:
!rm data/event_data_sherdog.csv

In [ ]:
import requests
import pandas as pd
import time
import random
from tqdm import tqdm
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
]

df = pd.read_csv('./data/event_urls_sherdog.csv')
df['Event_URL'] = "https://sherdog.com" + df['Event_URL'].astype(str)
event_dates = []

def fetch_event_date(url, session):
    headers = {"User-Agent": random.choice(user_agents)}
    for attempt in range(3):
        try:
            response = session.get(url, headers=headers, timeout=30)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                event_date_meta = soup.find('meta', itemprop='startDate')
                return event_date_meta['content'].strip() if event_date_meta else None
        except requests.exceptions.RequestException:
            time.sleep(random.uniform(1, 5))
    return "Error"

with requests.Session() as session:
    with ThreadPoolExecutor(max_workers=40) as executor:  # Increased max_workers for speed
        future_to_url = {executor.submit(fetch_event_date, url, session): url for url in df['Event_URL']}
        
        for future in tqdm(concurrent.futures.as_completed(future_to_url), total=len(df['Event_URL']), desc="Scraping Events", unit="Event"):
            try:
                event_dates.append(future.result())
            except Exception as e:
                event_dates.append("Error")

df['Event Date'] = event_dates
df.to_csv('./data/event_urls_sherdog.csv', index=False)

print("Event dates appended successfully.")
print(f"Total number of rows including the header in event_urls_sherdog.csv: {len(df)}")
print(f"Column names: {list(df.columns)}")

# user_agents = [
#     "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
#     "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
#     "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
#     "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
# ]
# df = pd.read_csv('./data/event_urls_sherdog.csv')
# df['Event_URL'] = "https://sherdog.com" + df['Event_URL'].astype(str)
# event_dates = []
# with ThreadPoolExecutor(max_workers=10) as executor:
#     for url in tqdm(df['Event_URL'], total=len(df['Event_URL']), desc="Scraping Events", unit="Event"):
#         for attempt in range(3):
#             try:
#                 headers = {"User-Agent": random.choice(user_agents)}
#                 response = requests.get(url, headers=headers, timeout=30)
#                 if response.status_code == 200:
#                     soup = BeautifulSoup(response.content, 'html.parser')
#                     event_date_meta = soup.find('meta', itemprop='startDate')
#                     event_dates.append(event_date_meta['content'].strip() if event_date_meta else None)
#                     break
#             except requests.exceptions.RequestException:
#                 time.sleep(random.uniform(1, 5))
#         else:
#             event_dates.append("Error")
# df['Event Date'] = event_dates
# df.to_csv('./data/event_urls_sherdog.csv', index=False)
# print("Event dates appended successfully.")
# print(f"Total number of rows including the header in event_urls_sherdog.csv: {len(df)}")
# print(f"Column names: {list(df.columns)}")


In [ ]:
# # Add Event Dates 

# df = pd.read_csv('./data/event_urls_sherdog.csv')
# df['Event_URL'] = "https://sherdog.com" + df['Event_URL'].astype(str)
# event_dates = []

# for url in tqdm(df['Event_URL'], desc="Scraping Events", unit="Event"):
#     for _ in range(3):
#         try:
#             response = session.get(url, timeout=30)
#             if response.status_code == 200:
#                 soup = BeautifulSoup(response.content, 'html.parser')
#                 event_date_meta = soup.find('meta', itemprop='startDate')
#                 event_dates.append(event_date_meta['content'].strip() if event_date_meta else None)
#                 break
#             else:
#                 time.sleep(5)
#         except requests.exceptions.RequestException:
#             time.sleep(5)
#     else:
#         event_dates.append("Error")

# df['Event Date'] = event_dates
# df.to_csv('./data/event_urls_sherdog.csv', index=False)
# print("Event dates appended successfully.")
# # df = pd.read_csv('./data/event_urls_sherdog.csv')
# print(f"Total number of rows including the header in event_urls_sherdog.csv: {len(df)}")
# print(f"Column names: {list(df.columns)}")

user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
]
df = pd.read_csv('./data/event_urls_sherdog.csv')
df['Event_URL'] = "https://sherdog.com" + df['Event_URL'].astype(str)
event_dates = []

with requests.Session() as session:
    session.headers.update({"User-Agent": random.choice(user_agents)})
    with ThreadPoolExecutor(max_workers=10) as executor:
        for url in tqdm(df['Event_URL'], total=len(df['Event_URL']), desc="Scraping Events", unit="Event"):
            for attempt in range(3):
                try:
                    session.headers.update({"User-Agent": random.choice(user_agents)})
                    response = session.get(url, timeout=30)
                    if response.status_code == 200:
                        soup = BeautifulSoup(response.content, 'html.parser')
                        event_date_meta = soup.find('meta', itemprop='startDate')
                        event_dates.append(event_date_meta['content'].strip() if event_date_meta else None)
                        break
                    time.sleep(random.uniform(1, 5))
                except requests.exceptions.RequestException:
                    time.sleep(random.uniform(1, 5))
            else:
                event_dates.append("Error")

df['Event Date'] = event_dates
df.to_csv('./data/event_urls_sherdog.csv', index=False)
print("Event dates appended successfully.")
print(f"Total number of rows including the header in event_urls_sherdog.csv: {len(df)}")
print(f"Column names: {list(df.columns)}")









# df = pd.read_csv('./data/event_urls_sherdog.csv')
# df['Event_URL'] = "https://sherdog.com" + df['Event_URL'].astype(str)
# event_dates = []

# with requests.Session() as session:
#     session.headers.update(headers)
#     with ThreadPoolExecutor(max_workers=10) as executor:
#         for url in tqdm(df['Event_URL'], total=len(df['Event_URL']), desc="Scraping Events", unit="Event"):
#             for attempt in range(3):
#                 try:
#                     response = session.get(url, timeout=30)
#                     if response.status_code == 200:
#                         soup = BeautifulSoup(response.content, 'html.parser')
#                         event_date_meta = soup.find('meta', itemprop='startDate')
#                         event_dates.append(event_date_meta['content'].strip() if event_date_meta else None)
#                         break
#                     time.sleep(2 ** attempt)
#                 except requests.exceptions.RequestException:
#                     time.sleep(2 ** attempt)
#             else:
#                 event_dates.append("Error")

# df['Event Date'] = event_dates
# df.to_csv('./data/event_urls_sherdog.csv', index=False)
# print("Event dates appended successfully.")


In [ ]:
!open data/fighter_info.csv
!open data/event_data_sherdog.csv
!open data/reach_dictionary.csv
!open data/github/master.csv
!open data/github/ufc_fighter_tott.csv